In [1]:
import os
import pandas as pd
import numpy as np

import re
from datetime import datetime
import warnings 

pd.set_option("display.precision", 6)
pd.options.display.float_format = '{:.6f}'.format

warnings.filterwarnings('ignore')

list_dataset = ["Lastfm", "Amazon-lb", "QK-video", "ML-10M"]
rerank_path = "../reranking/result/"

In [2]:
def load_base():

    result_file = [f for f in os.listdir("base") if "pickle" in f]
    results = {}

    for f in result_file:
        result = pd.read_pickle("base/"+f)
        f = f.replace(".pickle", "").replace("base_", "")
        results[f] = result
    base = pd.DataFrame(results)

    base = base[base.index.str.contains("@10")]
    base.columns = base.columns + "_-_x_x"
    return base


def load_rerank():
    reranking_result_file = [f for f in os.listdir(rerank_path) if "pickle" in f and "at10" in f and "rerank25" in f and "combmnz" in f]

    reranking_results = {}

    for f in reranking_result_file:
        result = pd.read_pickle(rerank_path+f)
        f = f.replace(".pickle", "").replace("base_", "")
        reranking_results[f] = result

    rerank = pd.DataFrame(reranking_results)
    return rerank

In [3]:
def combined_result_pure():
    base = load_base()
    rerank = load_rerank()
    combined_result = pd.concat([base, rerank], axis=1)
    combined_result = pd.DataFrame(combined_result.stack(dropna=False)).reset_index()
    return combined_result

In [4]:
df_combined_result_pure = combined_result_pure()
df_combined_result_pure

,level_0,level_1,0
0,HR@10,Amazon-lb_BPR_-_x_x,0.011442
1,HR@10,Amazon-lb_ItemKNN_-_x_x,0.045767
2,HR@10,Amazon-lb_MultiVAE_-_x_x,0.038902
3,HR@10,Amazon-lb_NCL_-_x_x,0.034325
4,HR@10,Lastfm_BPR_-_x_x,0.772876
...,...,...,...
859,HD@10,ML-10M_NCL_combmnz_at10_rerank25,0.256860
860,HD@10,QK-video_BPR_combmnz_at10_rerank25,0.564626
861,HD@10,QK-video_ItemKNN_combmnz_at10_rerank25,0.559643
862,HD@10,QK-video_MultiVAE_combmnz_at10_rerank25,0.535491


In [5]:



selected_index = pd.Index([
    'HR', 'MRR', 'P', 'MAP', 'R', 'NDCG', 
    'Jain_our', 'QF_our', 'Ent_our', 'FSat_our', 
    'Gini_our',
    'IBO_our',
    'IWO_our', 
    'IAA_true_ori',
    'IFD_div', 'IFD_mul',
    'HD', "MME_ori",
    'II-F_ori', 'AI-F_ori'
])

def modify_combined_result(combined_result,selected_index):
    combined_result = combined_result.copy()

    combined_result[["measures","k"]] = combined_result["level_0"].str.split("@", expand=True)
    combined_result[["dataset", "model", "reranking", "dummy", "dummy2"]] = combined_result.level_1.str.split("_", expand=True)
    combined_result = combined_result.drop(columns=["level_0", "level_1", "dummy", "dummy2"])
    combined_result["k"] = combined_result["k"].apply(int)
    combined_result["reranking"] = combined_result["reranking"].str.replace("combmnz", "CM")
    combined_result = combined_result.set_index(["dataset", "measures","model", "reranking"]).drop(columns="k").unstack(2).droplevel(0, axis=1)
    combined_result = combined_result.reindex(list_dataset, axis=0, level=0)


    combined_result = combined_result.reindex(selected_index, level=1)
    return combined_result


In [6]:
combined_result = modify_combined_result(df_combined_result_pure,selected_index=selected_index)
combined_result

model                           BPR  ItemKNN  MultiVAE      NCL
dataset measures reranking                                     
Lastfm  HR       -         0.772876 0.764706  0.778322 0.793028
                 CM        0.587146 0.580610  0.523420 0.571351
        MRR      -         0.491556 0.483544  0.476367 0.502564
                 CM        0.280363 0.270167  0.231551 0.259868
        P        -         0.178159 0.172059  0.175763 0.184041
...                             ...      ...       ...      ...
ML-10M  MME_ori  CM        0.000635 0.000845  0.001450 0.000687
        II-F_ori -         0.000351 0.000353  0.000354 0.000351
                 CM        0.000357 0.000355  0.000356 0.000357
        AI-F_ori -         0.000008 0.000023  0.000072 0.000013
                 CM        0.000003 0.000008  0.000034 0.000004

[160 rows x 4 columns]

In [7]:
now = datetime.now()
time = str(now.strftime("%Y-%m-%d_%H%M%S"))
combined_result.reset_index().to_csv(f"combined_base/csv_combined_result_{time}.csv")
time